#  This notebook is for the Capstone Project of the Coursera course Applied Data Science
** First the postcodes of Toronto will be scraped from a wikipeda page. This data will be stored and processed into a Data Frame.**

In [518]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

**Get the HTML code from the wikipedia website with the postcodes**

In [519]:
#get the html code from the respective website
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#store the html file in the soup variable with the lxml parser
soup = BeautifulSoup(page, 'lxml')


**Initialize some empty lists**

In [521]:
# initialize some variables
columns_headers = []
values_column1 = []
values_column2 = []
values_column3 = []
filter_DF =[]


** Scrape the webpage. We will get the name for the columns as well as the data for each row from the HTML code**

In [489]:
# get the relevent div from the whole soup
source = soup.find('div', class_='mw-parser-output')

In [490]:
# search for all entries in the first row of the table (the headings)
for tr in source.find_all('tr')[0].find_all('th'):
    columns_headers.append(tr.text)

In [491]:
# because the last heading has a \n at the end, we have the cut it away
columns_headers[2] = columns_headers[2][0:-1]

In [492]:
# loop through the table to get all the entries for the columns
for td in source.find_all('td')[::3]:
    values_column1.append(td.text)
for td in source.find_all('td')[1::3]:
    values_column2.append(td.text)    
for td in source.find_all('td')[2::3]:
    values_column3.append(td.text)    
    

In [522]:
# because there is some extra code that is not part of the table we have to get rid of the irrelevant entries
relevant_number_rows = values_column1.index('M9Z')
values_column1 = values_column1[0:relevant_number_rows+1]
values_column2 = values_column2[0:relevant_number_rows+1]
values_column3 = values_column3[0:relevant_number_rows+1]

#again in column 3 there is a /n at the end that has be truncated
for i,value in enumerate(values_column3):
    values_column3[i-1] = values_column3[i-1][0:-1]

ValueError: 'M9Z' is not in list

**We will now create the data frame with the data we have obtained**
**Then we will wrangle the data, so we can better process it in a later stage**

In [494]:
# create a data frame
df_values = np.array([values_column1,values_column2,values_column3]).transpose()
Neighborhoods_CN = pd.DataFrame(data = df_values,columns = columns_headers)


In [495]:
#drop all the rows with 'Borough' = Not assinged
Neighborhoods_CN['Borough'].replace(to_replace='Not assigned',value = np.nan,inplace=True)
Neighborhoods_CN.dropna(inplace=True)
Neighborhoods_CN.reset_index(drop = True,inplace = True)


In [496]:
unique_postcodes = Neighborhoods_CN['Postcode'].unique().tolist()


In [497]:
# getting a list with the boolean values for unique postcodes
length = len(Neighborhoods_CN['Postcode'])

for i in range(0,length):
    filter_DF.append(Neighborhoods_CN['Postcode'][i] in unique_postcodes)
    if Neighborhoods_CN['Postcode'][i] in unique_postcodes:     
        unique_postcodes.pop(unique_postcodes.index(Neighborhoods_CN['Postcode'][i]))
        
filter_DF_reversed = list(~np.array(filter_DF))

In [498]:
# creating two new data frames. The first one with single postcodes.
# The second one with the rest of the intial data frame
Neighborhoods_CN_filtered1 = Neighborhoods_CN[filter_DF]
Neighborhoods_CN_filtered2 = Neighborhoods_CN[filter_DF_reversed]

for val in Neighborhoods_CN_filtered2['Postcode'].values.tolist():
    for i in Neighborhoods_CN_filtered2[Neighborhoods_CN_filtered2['Postcode']==val].index:
        row = Neighborhoods_CN_filtered1[Neighborhoods_CN_filtered1['Postcode']==val].index[0]
        val_to_add =  Neighborhoods_CN_filtered2.loc[i,'Neighbourhood']
        Neighborhoods_CN_filtered1.loc[row,'Neighbourhood'] = Neighborhoods_CN_filtered1.loc[row,'Neighbourhood'] + ', ' + val_to_add

C:\Users\netze\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\netze\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [513]:
# now we append the Neighbourhood of the second data frame to the first one for the respective postcodes 
for i in Neighborhoods_CN_filtered1[Neighborhoods_CN_filtered1['Neighbourhood'] == 'Not assigned']['Borough'].index:
    val = Neighborhoods_CN_filtered1.loc[i,'Borough']
    Neighborhoods_CN_filtered1.replace(to_replace='Not assigned', value=val, inplace =True)

C:\Users\netze\Anaconda3\lib\site-packages\pandas\core\frame.py:3795: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [517]:
Neighborhoods_CN_filtered1.shape

(103, 3)